In [ ]:
import os
import pickle
import torch
import optuna
import matplotlib.pyplot as plt
import numpy as np

from Project.VisualFeaturesBranches.ObjectDetection.ObjectDetectionNetworkUtils import get_config_score
from Project.Misc.Misc import crop_background

In [ ]:
def objective(trial):
    return get_config_score(
        {
        'lr': trial.suggest_loguniform('learning_rate', 1e-6, 1e-3),
        'weight_decay': trial.suggest_loguniform('weight_decay', 1e-5, 1e-1),
        'trainable_layers': trial.suggest_int('trainable_layers', 1, 6),
        'size': 1200,
        'p_horizontal_flip': 0,
        'p_vertical_flip': 0,
        'brightness_jitter': 0,
        'contrast_jitter': 0,
        'saturation_jitter': 0,
        'hue_jitter': 0,
        'p_grayscale': 0,
        'batch_size': 1,
        'num_workers': 0,
        'device': torch.device('cuda'),
        'average_span': 5,
        'nr_epochs': 10,
        'report': False,
        'small_model': True,
        'curriculum': False
        }, trial
    )

if os.path.exists(study_path := 'study_without_augmentations.pkl'):
    with open(study_path, 'rb') as f:
        study = pickle.load(f)
else:
    study = optuna.create_study(direction="maximize", study_name='No_augmentations', pruner=optuna.pruners.SuccessiveHalvingPruner(min_resource=1, reduction_factor=4), sampler=optuna.samplers.TPESampler(seed=0, multivariate=True, group=True, n_startup_trials=10))
    study.optimize(objective, n_trials=40, show_progress_bar=True)

    with open(study_path, 'wb+') as f:
        pickle.dump(study, f)

In [ ]:
study.best_trial.params

In [ ]:
optuna.visualization.matplotlib.plot_optimization_history(study, target_name='F1 Score')

In [ ]:
optuna.visualization.plot_intermediate_values(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study, target_name='F1 Score')

In [ ]:
optuna.visualization.plot_param_importances(study, evaluator=optuna.importance.FanovaImportanceEvaluator(n_trees=10000, max_depth=2**8))

In [ ]:
ideal_trainable_layers = study.best_trial.params['trainable_layers']
def objective(trial):
    hue_jitter = trial.suggest_loguniform('hue_jitter', 0.001, 0.5)
    return get_config_score(
        {
        'lr': trial.suggest_loguniform('learning_rate', 1e-6, 1e-3),
        'weight_decay': trial.suggest_loguniform('weight_decay', 1e-5, 1e-1),
        'trainable_layers': ideal_trainable_layers,
        'size': 1200,
        'p_horizontal_flip': trial.suggest_uniform('p_horizontal_flip', 0, 0.5),
        'p_vertical_flip': trial.suggest_uniform('p_vertical_flip', 0, 0.5),
        'brightness_jitter': trial.suggest_uniform('brightness_jitter', 0.001, 1),
        'contrast_jitter': trial.suggest_loguniform('contrast_jitter', 0.001, 1),
        'saturation_jitter': trial.suggest_loguniform('saturation_jitter', 0.001, 1),
        'hue_jitter': (-hue_jitter, hue_jitter),
        'p_grayscale': trial.suggest_uniform('p_grayscale', 0, 0.5),
        'batch_size': 1,
        'num_workers': 1,
        'device': torch.device('cuda'),
        'average_span': 5,
        'nr_epochs': 10,
        'report': False,
        'small_model': True,
        'curriculum': True
        }, trial
    )
if os.path.exists(study_path := 'study_with_augmentations.pkl'):
    with open(study_path, 'rb') as f:
        study = pickle.load(f)
else:
    # Note: min_resource -> It is not expected that the augmented models perform better in the first epochs then the unaugmented ones
    study = optuna.create_study(direction="maximize", study_name='With_augmentations', pruner=optuna.pruners.SuccessiveHalvingPruner(min_resource=5), sampler=optuna.samplers.TPESampler(seed=0, multivariate=True, group=True, n_startup_trials=10))
    study.optimize(objective, n_trials=40, show_progress_bar=True)
    with open(study_path, 'wb+') as f:
        pickle.dump(study, f)

In [ ]:
study.best_trial

In [ ]:
optuna.visualization.matplotlib.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_intermediate_values(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_param_importances(study, evaluator=optuna.importance.FanovaImportanceEvaluator(n_trees=10000, max_depth=2**8), target_name='F1 Score', params=['brightness_jitter', 'hue_jitter', 'p_vertical_flip', 'p_horizontal_flip', 'contrast_jitter', 'p_grayscale', 'saturation_jitter'])

In [ ]:
# Note: The above results can differ in each run, so these numbers might not always be exactly the same.
# A matplotlib graph was made so that the labels could be changed
scores = {'Brightness Jitter': 0.43857,
          'Hue Jitter': 0.23048,
          'P vertical Flip': 0.0855,
          'Contrast Jitter': 0.08055,
          'P horizontal Flip': 0.0723,
          'P Grayscale': 0.06114,
          'Saturation Jitter': 0.03144
          }

fig, axes = plt.subplots()
title = 'Importance of Augmentation Techniques for F1 Score'
axes.set_title(title, fontsize=24)
plt.xticks(rotation=45, ha='right', fontsize=12)
axes.bar(scores.keys(), scores.values())

for i,j in zip(np.arange(len(scores.keys())), scores.values()):
    axes.annotate(str(j)[0:5],xy=(i-0.3,j-0.0225), color='white', weight='bold', fontsize=24)
fig.set_figheight(12)
fig.set_figwidth(17)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.savefig(path:='importance_of_augmentations')
crop_background(os.path.abspath(path + '.png'))